## Chapter 14 - Exercise 9

Build a CNN from scratch and try and achieve the highest possible accuracy on MNIST.

In [1]:
import tensorflow.keras as tfk
import tensorflow as tf

2024-01-28 03:02:39.732843: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-28 03:02:39.775890: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 03:02:39.775921: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 03:02:39.777061: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 03:02:39.784418: I tensorflow/core/platform/cpu_feature_guar

In [5]:
(X_train_val, y_train_val), (X_test, y_test) = tfk.datasets.mnist.load_data()
X_train_val = X_train_val / 255.
X_train, X_val = X_train_val[:-5000], X_train_val[-5000:]
y_train, y_val = y_train_val[:-5000], y_train_val[-5000:]
X_test = X_test / 255.

X_train = X_train.reshape(-1, 28, 28, 1)
X_val = X_val.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

tfk.backend.clear_session()
tf.random.set_seed(42)

data_augmentation = tfk.Sequential([
    tfk.layers.RandomFlip(mode="horizontal", seed=42),
    tfk.layers.RandomRotation(factor=0.05, seed=42),
    tfk.layers.RandomContrast(factor=0.2, seed=42)
    ])

model = tfk.models.Sequential()
model.add(data_augmentation)
for _ in range(3):
    model.add(tfk.layers.Conv2D(filters=64, kernel_size=3, padding="same",
                                activation="swish", kernel_initializer="he_normal"))
model.add(tfk.layers.MaxPool2D())
for _ in range(2):
    model.add(tfk.layers.Conv2D(filters=128, kernel_size=3, padding="same",
                                activation="swish", kernel_initializer="he_normal"))
model.add(tfk.layers.MaxPool2D())
model.add(tfk.layers.Conv2D(filters=256, kernel_size=3, padding="same",
                            activation="swish", kernel_initializer="he_normal"))
model.add(tfk.layers.Flatten())
model.add(tfk.layers.Dropout(0.25))
model.add(tfk.layers.Dense(256, activation="swish", kernel_initializer="he_normal"))
model.add(tfk.layers.Dropout(0.5))
model.add(tfk.layers.Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])

In [7]:
reduce_lr_cb = tfk.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.3, verbose=1,
                                               patience=2, min_lr=1e-8)
model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val),
          callbacks=[reduce_lr_cb])

Epoch 1/20


2024-01-28 03:03:52.360645: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-28 03:03:53.384128: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-28 03:03:53.515181: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-28 03:03:55.186742: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f6c1e8401a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-28 03:03:55.186777: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-01-28 03:03:55.199624: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1706411035.329824     231 device_compiler.h:186] Compiled

1719/1719 [==============================] - 30s 13ms/step - loss: 0.3101 - accuracy: 0.9105 - val_loss: 0.1349 - val_accuracy: 0.9710 - lr: 0.0010
Epoch 2/20
1719/1719 [==============================] - 22s 13ms/step - loss: 0.2109 - accuracy: 0.9448 - val_loss: 0.0843 - val_accuracy: 0.9796 - lr: 0.0010
Epoch 3/20
1719/1719 [==============================] - 22s 13ms/step - loss: 0.1830 - accuracy: 0.9538 - val_loss: 0.0786 - val_accuracy: 0.9824 - lr: 0.0010
Epoch 4/20
1719/1719 [==============================] - 22s 13ms/step - loss: 0.1611 - accuracy: 0.9587 - val_loss: 0.0685 - val_accuracy: 0.9860 - lr: 0.0010
Epoch 5/20
1719/1719 [==============================] - 22s 13ms/step - loss: 0.1963 - accuracy: 0.9564 - val_loss: 0.0871 - val_accuracy: 0.9818 - lr: 0.0010
Epoch 6/20
1719/1719 [==============================] - 22s 13ms/step - loss: 0.1485 - accuracy: 0.9637 - val_loss: 0.0637 - val_accuracy: 0.9846 - lr: 0.0010
Epoch 7/20
1719/1719 [==============================] - 2

In [8]:
model.evaluate(X_val, y_val)

157/157 [==============================] - 1s 4ms/step - loss: 0.0430 - accuracy: 0.9926


[0.0429859422147274, 0.9926000237464905]

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 64)        0         
 D)                                                              
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 128)       73856     
                                                      